# 导入模块

In [1]:
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time
from apscheduler.schedulers.blocking import BlockingScheduler
import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta, datetime 
import re
import os

# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:

pd.set_option('display.max_columns', None)
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason

,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.order_type, tpmn.name 机型
,tp.classify_id
from  db_digua_business.t_order  om

left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id

-- 商品表
left join db_digua_business.t_product tp on tp.id=tod.product_id -- 8942.97MB
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id

-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- AND tmer.shop_type !=2  -- 剔除商家
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -1 month )               -- 近15天数据
-- and  om.create_time < DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-10-24'
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-10-30'

;
'''

# df_order = query(sql1)
# df_order.shape

# 数据处理函数

In [5]:
def process_data(df_order):
    # 复制数据
    df = df_order.copy()
    # df = df[df.type!=4]
    
    # 处理日期
    df["下单日期"]=df["create_time"].dt.date
    df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
    print(df["下单日期"].unique())
    df["月份"]=df["下单日期"].dt.month
    df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    
    # 处理备注信息
    df = df[df['sku_attributes'].notnull()]
    df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
    df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
    df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
    df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
    df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
    df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
    df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

    # 处理备注信息total_describes,cancel_reason
    df['取消原因2'] = df['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
    def getcolor(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="颜色":
                return color_list[j]["value"]
    df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)


    def getneicun(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="内存":
                return color_list[j]["value"]

    df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
    df.loc[:,'机型内存'] = df['机型'] +"_"+ df['内存']

    df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")


    df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
    df['免人审'] = np.where((df.is_vip=='1')&(df.status_result=='0'), 1, 0)
    
    df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
    
    # 订单去重
    df = clean.order_drop_duplicates(df)
    
    # 定义状态
    df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
    df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
    df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
    
    df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
    
    # 去重订单明细导出
    today = datetime.today() 
    t_date = today.date().strftime('%Y-%m-%d')
    
    df_j = df[df["进件"]=="进件"]
    
    df["待审核"] = np.where(df["审核状态"] == '待审核', 1, 0)
    df["前置拦截"] = np.where(df["审核状态"] == '前置拦截', 1, 0)
    df["人审拒绝"] = np.where(df["审核状态"] == '人审拒绝', 1, 0)
    df["客户取消"] = np.where(df["审核状态"] == '客户取消', 1, 0)
    df["无法联系"] = np.where(df["审核状态"] == '无法联系', 1, 0)
    df["是否进件"] = np.where(df["进件"] == '进件', 1, 0)
    df["是否出库"] = np.where(df["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)

    df["进件前取消"] = np.where(df["审核状态"] == '进件前取消', 1, 0)
    df['是否出库'] = np.where((df['人审拒绝'] == 0) & (df['客户取消'] == 0) & (df['无法联系'] == 0) & (df['待审核'] == 0) & (df['是否出库'] == 1), 1, 0)
    df["出库前风控强拒"] = np.where((df["审核状态"] == '出库前风控强拒') & (df['是否出库'] == 0), 1, 0)
    df["机审强拒"] = np.where((df["审核状态"] == '机审强拒') & (df['是否出库'] == 0), 1, 0)
    
    # 出库数据
    # df_cks = df[df["是否出库"]==1]
    
    # 导入出库台账
    # f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
    # df_ck = pd.read_excel(f_path_ck,sheet_name="2025") 
    
    # dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")
    # dfck.drop_duplicates(subset=["订单号"],inplace=True)
    
    # dfck[dfck["status2"]=="已退款"]

    # dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

    # dfck[dfck["status2"]=="已退款"]

    # # 删除 露营设备 出库 
    # try:
    #     dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
    # except:
    #     dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
        
    # # 将序列号转换为日期
    # base_date = pd.to_datetime('1899-12-30')  # Excel的基准日期
    # dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')
    
    return df


# 提取订单风控数据

# 合并数据

In [64]:
# # df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
# df = df_order.copy()
# df.order_id.notnull().sum()

# 处理日期

In [65]:

# df["下单日期"]=df["create_time"].dt.date
# df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
# df["下单日期"]
# df["月份"]=df["下单日期"].dt.month
# df.下单日期.unique()

In [66]:
# df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
# df['拒绝理由'].value_counts()

# 处理备注信息

In [67]:
# df = df[df['sku_attributes'].notnull()]

In [68]:

# df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
# df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
# df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
# df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
# df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
# df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
# df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
# df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

# df['取消原因2'] = df['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# def getcolor(s):
#     color_list=json.loads(s)
#     for j in range(0,len(color_list)):
#         if color_list[j]["key"]=="颜色":
#             return color_list[j]["value"]
# df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
# df["颜色"].value_counts()  

# # %%
# def getneicun(s):
#     color_list=json.loads(s)
#     for j in range(0,len(color_list)):
#         if color_list[j]["key"]=="内存":
#             return color_list[j]["value"]

# df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
# df["内存"].value_counts() 

# df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
# df["进件"].value_counts()

# # %%
# df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [69]:
# df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
# df['免审'].value_counts()
# df[df.来源渠道=='搜索侠客行-首页'].head()
# df.来源渠道.unique()

In [70]:

# df['免人审'] = np.where((df.is_vip=='1')&(df.status_result=='0'), 1, 0)
# df['免人审'].value_counts()


# 渠道归属

In [71]:
# df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
# df.归属渠道.value_counts()

In [72]:
# df[df['来源渠道']=='搜索侠客行-首页'].shape

# 订单去重

In [73]:
# dict_status_code = {
#     "订单取消": 1,
#     "待支付": 2,
#     "已退款": 3, 
#     "待审核": 4,
#     "待发货": 5,
#     "待收货": 6,
#     "租赁中": 7,
#     "已完成": 8
# }
# df["状态编码"]=df["status2"].map(dict_status_code)


# df.sort_values(by=["下单日期","状态编码"],inplace=True)


# #删除重复单号
# df.drop_duplicates(subset=["order_id"],inplace=True)
# #删除身份证空值行
# df.dropna(subset=["id_card_num"],axis=0,inplace=True)
# #去刷单订单
# df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
# df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
# df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
# df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
# df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
# #删除身份证空值行
# df.dropna(subset=["status2"],axis=0,inplace=True)


# #删除重复订单
# # df.drop_duplicates(subset=["order_id"],inplace=True)
# df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


# df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
# ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆'
# ,'黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍'])].index,inplace=True)

In [74]:
# df[df['来源渠道']=='搜索侠客行-首页'].shape

# 定义状态

In [75]:
# # 判断 前置拦截   机审强拒   出库前风控强拒
# df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
#                         |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [76]:
# df[df['是否前置拦截'] ==1].拒绝理由.value_counts()
# df[df['出库前风控强拒'] ==1].拒绝理由.value_counts()
# df[(df['是否前置拦截'] ==1) & (df.活动名称.str.contains('抖音'))].拒绝理由.value_counts()
# df[df['机审强拒'] ==1].拒绝理由.value_counts()

In [77]:
# def reject_type(a,b,c,d,e,f,g,h,i,j):
#     if h==1 and b=="未进件":
#         return "前置拦截"
#     elif i==1 and b=="进件":
#         return "机审强拒"    
#     #抖音渠道
#     # elif h==1 and b=="进件":
#     #     return "机审强拒"  
#     elif  b=="进件" and c is not np.nan:
#         return "人审拒绝"
#     elif  b=="进件" and d is not np.nan:
#         return "客户取消"
#     elif  b=="进件" and f is not np.nan:
#         return "无法联系"
#     elif  b=="进件" and j==1 :
#         return "出库前风控强拒"
#     elif  b=="进件" and "待审核" in e:
#         return "待审核"
#     elif  b=="进件" and "待发货" in e:
#         return "出库"
#     elif  b=="进件" and "待收货" in e:
#         return "出库"
#     elif b=="进件" and "租赁中" in e:
#         return "出库"
#     elif  b=="进件" and "已完成" in e:
#         return "出库"
#     elif a is np.nan and b=="未进件":
#         return "未进件"
#     elif  b=="未进件":
#         return "进件前取消"
#     else :
#         return e

# df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
# df["审核状态"].value_counts()  

# 去重订单明细导出

In [78]:
# from datetime import datetime  
# today = datetime.today() 
# t_date = today.date().strftime('%Y-%m-%d')
# t_date

# 导入服务订单

In [79]:
# df["下单时段"]=df["create_time"].astype(str).str[:14]

# # %%
# df_j = df[df["进件"]=="进件"]
# df_j.shape

In [80]:




# df["待审核"] = np.where(df["审核状态"] == '待审核', 1, 0)
# df["前置拦截"] = np.where(df["审核状态"] == '前置拦截', 1, 0)
# df["人审拒绝"] = np.where(df["审核状态"] == '人审拒绝', 1, 0)
# df["客户取消"] = np.where(df["审核状态"] == '客户取消', 1, 0)
# df["无法联系"] = np.where(df["审核状态"] == '无法联系', 1, 0)
# df["是否进件"] = np.where(df["进件"] == '进件', 1, 0)
# df["是否出库"] = np.where(df["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)

# df["进件前取消"] = np.where(df["审核状态"] == '进件前取消', 1, 0)
# df['是否出库'] = np.where((df['人审拒绝'] == 0) & (df['客户取消'] == 0) & (df['无法联系'] == 0) & (df['待审核'] == 0) & (df['是否出库'] == 1), 1, 0)
# df["出库前风控强拒"] = np.where((df["审核状态"] == '出库前风控强拒') & (df['是否出库'] == 0), 1, 0)
# df["机审强拒"] = np.where((df["审核状态"] == '机审强拒') & (df['是否出库'] == 0), 1, 0)



In [81]:
# df_cks = df[df["是否出库"]==1]

# 导入出库台账数据

In [82]:
# #读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2025") 
# df_ck.shape

In [83]:
# df_ck['溢价费订单'] = "A"+df_ck[df_ck['备注'].str.contains(pat='溢价费',regex=False)==True]['备注'].str.extract('(\d+.\d+)')
# df_yijia = df_ck[df_ck['溢价费订单'].str.len()>=16][['溢价费订单',"已付金额"]]
# df_yijia=df_yijia.rename(columns={'溢价费订单':"单号","已付金额":"溢价费"})
# df_yijia_ck = pd.merge(df_ck,df_yijia,left_on="订单号", right_on="单号",how="left")
# dfck = pd.merge(df_yijia_ck,df,left_on="订单号", right_on="order_number")

# dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")

# dfck.drop_duplicates(subset=["订单号"],inplace=True)
# dfck.shape#.to_excel(f"C:/Users/Administrator/Desktop/报表模板/出库台账最终整理/11月出库补充1220.xlsx")

In [84]:
# dfck[dfck["status2"]=="已退款"]

# dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# dfck[dfck["status2"]=="已退款"]

# # 删除 露营设备 出库 
# try:
#     dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
# except:
#     dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
    
# # 将序列号转换为日期
# base_date = pd.to_datetime('1899-12-30')  # Excel的基准日期
# dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')


# 搜索灯火

In [85]:
def group_data(df0, classify):
    df_group = df0.groupby(classify).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
    df_group.rename(columns={'order_id':'去重订单数', '是否进件': '进件数', '是否出库': '出库'},inplace=True )

    df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

    df_group["进件前取消率"] = df_group["进件前取消"]/df_group["去重订单数"]
    df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x:format(x,".2%"))

    df_group['预授权通过率'] = df_group["进件数"]/df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

    df_group["强拒比例"] = df_group["机审强拒"]/df_group['进件数']
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["机审通过件"] = df_group["进件数"]-df_group["机审强拒"]

    df_group["风控通过件"] = df_group["进件数"]-df_group["机审强拒"]-df_group["人审拒绝"]
    df_group["风控通过率"] = df_group["风控通过件"]/df_group['进件数']
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

    df_group["取消率"] = df_group["客户取消"]/df_group['进件数']
    df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

    df_group["无法联系占比"]=df_group["无法联系"]/df_group['进件数']
    df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

    df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group['进件数']
    df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df_group["出库前强拒比例"]=df_group["出库前风控强拒"]/df_group['进件数']
    df_group["出库前强拒比例"]=df_group["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["进件出库率"]=df_group["出库"]/df_group['进件数']
    df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

    df_group["订单出库率"]=df_group["出库"]/df_group["去重订单数"]
    df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

    return df_group

In [86]:
# df_j['merchant_name'].drop_duplicates()

In [87]:
# df_j['订单号'] = df_j['order_number']
# df_j['商品型号'] = df_j['product_name']
# df_j['买断金额'] = df_j['new_actual_money']
# df_j['总租金'] = df_j['all_money']
# df_j['订单状态'] = df_j['status2']

# df_cks['订单号'] = df_cks['order_number']
# df_cks['商品型号'] = df_cks['product_name']
# df_cks['总租金'] = df_cks['all_money']
# df_cks['订单状态'] = df_cks['status2']
# df_cks['买断金额'] = df_cks['new_actual_money']
# Today1 = str(datetime.now().strftime('%Y%m%d%H'))

# 灯火渠道运营（晋辉需求）

In [88]:
df_order = query(sql1)
# 数据处理
df = process_data(df_order)
classify_id_list = [125,126,127,185]
df.loc[:, '设备种类'] = np.where((df.classify_id.isin(classify_id_list)), "电脑", "手机")
# 获取各个渠道的数据
# df_ss1 = df[df['来源渠道']=='搜索侠客行-首页']
df_bdqy = df[df['来源渠道']=='邦道-全域自定义电脑']

df_bdqy_group = group_data(df_bdqy, ['下单日期', '设备种类'])
df_bdqy_group = df_bdqy_group[['去重订单数', '进件数', '出库', '进件出库率', '订单出库率']]
df_bdqy_group

<DatetimeArray>
['2025-10-17 00:00:00', '2025-10-18 00:00:00', '2025-10-19 00:00:00',
 '2025-10-20 00:00:00', '2025-10-21 00:00:00', '2025-10-22 00:00:00',
 '2025-10-23 00:00:00', '2025-10-24 00:00:00', '2025-10-25 00:00:00',
 '2025-10-26 00:00:00', '2025-10-27 00:00:00', '2025-10-28 00:00:00',
 '2025-10-29 00:00:00', '2025-10-30 00:00:00', '2025-10-31 00:00:00',
 '2025-11-01 00:00:00', '2025-11-02 00:00:00', '2025-11-03 00:00:00',
 '2025-11-04 00:00:00', '2025-11-05 00:00:00', '2025-11-06 00:00:00',
 '2025-11-07 00:00:00', '2025-11-08 00:00:00', '2025-11-09 00:00:00',
 '2025-11-10 00:00:00', '2025-11-11 00:00:00', '2025-11-12 00:00:00',
 '2025-11-13 00:00:00', '2025-11-14 00:00:00', '2025-11-15 00:00:00',
 '2025-11-16 00:00:00', '2025-11-17 00:00:00']
Length: 32, dtype: datetime64[ns]


,,去重订单数,进件数,出库,进件出库率,订单出库率
下单日期,设备种类,,,,,
2025-10-26,电脑,1,0,0,nan%,0.00%
2025-10-28,电脑,1,1,0,0.00%,0.00%
2025-10-29,手机,1,0,0,nan%,0.00%
2025-10-31,手机,1,0,0,nan%,0.00%


In [89]:
df.来源渠道.unique()

array(['未知渠道', '顶部搜索框的搜索结果页', '芝麻信用', '搜索-租苹果手机', '八派信息', '搜索下挂-商品',
       '支付宝直播', '小程序商家消息（服务提醒）', '京东出游季', '芝麻信用频道', '搜索直达banner',
       '邦道月活BD22', '支付宝客户端首页', '搜索-租手机', '其他渠道场景渠道。', '我的小程序入口',
       '生活号-J-苹果旗舰', '生活号-商品组件', '搜索邦道-小程序推广', '单人聊天会话中的小程序消息卡片（分享）',
       '搜索侠客行-苹果17', '搜索侠客行-核心词', '支群-社群专享', '宜品花', '邦道-自定义苹果17', '派金花',
       '公域搜索带券', '搜索-租手机苹果', '搜索-租二手手机', '付费流量(通过商家数字推广平台，灯火等投放的广告)',
       '首页十二宫格及更多', '支群2-社群专属福利', '扫描二维码', '搜索-3C数码租赁', '租iPhone17',
       '搜索-租笔记本电脑', '生活号-J-电脑设备', '系统桌面图标', '社群-每日信息首页', '赛租机', '卡包\r\n',
       '九州信息', '第三方 APP（如钉钉）打开，在跳转链接中传入访问来源参数：', '订阅消息', '邦道-销售线索-电脑',
       '搜索-租机', '搜索-租运动相机', '侠客行自定义苹果17', '搜索-租华为手机', '搜索侠客行-苹果新词',
       '邦道-搜索-苹果17', '生活号-banner', '搜索-租二手苹果', '曙光计划', '搜索-租PS游戏机',
       '邦道月活BD23', '搜索-租游戏手机', '搜索-租电脑', '搜索-租相机', '会员卡-进入小程序',
       '搜索-租真我手机', '搜索-租无人机', '美团', '邦道-小程序-电脑', '搜索-租摄影手机', '邦道-小程序-手机',
       '侠客行全域租物', '租vivo新机', '支付宝 push 消息（同1014）', '搜索-租苹果耳机',
       '邦道-全域自定义电脑', '京享租右卡', '租苹果17

In [90]:
df_xcx = df[df['来源渠道']=='邦道-小程序-电脑']

df_xcx_group = group_data(df_xcx, ['下单日期', '设备种类'])
df_xcx_group = df_xcx_group[['去重订单数', '进件数', '出库', '进件出库率', '订单出库率']]
df_xcx_group

去重订单数  进件数  出库  进件出库率  订单出库率
下单日期       设备种类                              
2025-10-24 手机        4    1   0  0.00%  0.00%
           电脑        1    0   0   nan%  0.00%
2025-10-25 手机        3    0   0   nan%  0.00%
2025-10-26 手机        8    1   0  0.00%  0.00%
2025-10-27 手机        5    1   0  0.00%  0.00%
2025-10-28 手机       10    2   0  0.00%  0.00%
2025-10-29 手机       10    1   0  0.00%  0.00%
2025-10-30 手机        6    4   0  0.00%  0.00%
2025-10-31 手机        4    4   0  0.00%  0.00%
2025-11-01 手机        2    0   0   nan%  0.00%
2025-11-02 手机        7    0   0   nan%  0.00%
2025-11-03 手机        6    1   0  0.00%  0.00%
2025-11-04 手机        1    0   0   nan%  0.00%

In [91]:
df_xsxs = df[df['来源渠道']=='邦道-销售线索-电脑']

df_xsxs_group = group_data(df_xsxs, ['下单日期', '设备种类'])
df_xsxs_group = df_xsxs_group[['去重订单数', '进件数', '出库', '进件出库率', '订单出库率']]
df_xsxs_group = df_xsxs_group.reset_index()
df_xsxs_group.loc['汇总', :] = df_xsxs_group.select_dtypes(include=[np.number]).sum()
df_xsxs_group

,下单日期,设备种类,去重订单数,进件数,出库,进件出库率,订单出库率
0,2025-10-18,电脑,1.0,1.0,0.0,0.00%,0.00%
1,2025-10-22,手机,1.0,0.0,0.0,nan%,0.00%
2,2025-10-23,手机,2.0,1.0,0.0,0.00%,0.00%
3,2025-10-23,电脑,1.0,0.0,0.0,nan%,0.00%
4,2025-10-25,手机,2.0,0.0,0.0,nan%,0.00%
5,2025-10-25,电脑,2.0,0.0,0.0,nan%,0.00%
6,2025-10-26,手机,2.0,0.0,0.0,nan%,0.00%
7,2025-10-27,手机,1.0,0.0,0.0,nan%,0.00%
汇总,NaT,NaN,12.0,2.0,0.0,NaN,NaN


In [92]:
df_xsxs2 = df[df['来源渠道']=='邦道-销售线索2电脑']

df_xsxs2_group = group_data(df_xsxs2, ['下单日期', '设备种类'])
df_xsxs2_group = df_xsxs2_group[['去重订单数', '进件数', '出库', '进件出库率', '订单出库率']]
df_xsxs2_group = df_xsxs2_group.reset_index()
df_xsxs2_group.loc['汇总', :] = df_xsxs2_group.select_dtypes(include=[np.number]).sum()
df_xsxs2_group

,下单日期,设备种类,去重订单数,进件数,出库,进件出库率,订单出库率
汇总,NaT,NaN,0.0,0.0,0.0,0.0,0.0


In [93]:
df_xcxsj = df[df['来源渠道']=='邦道-小程序-手机']

df_xcxsj_group = group_data(df_xcxsj, ['下单日期', '设备种类'])
df_xcxsj_group = df_xcxsj_group[['去重订单数', '进件数', '出库', '进件出库率', '订单出库率']]
# df_xcxsj_group.loc['汇总', :] = df_xcxsj_group.select_dtypes(include=[np.number]).sum()
df_xcxsj_group

去重订单数  进件数  出库   进件出库率  订单出库率
下单日期       设备种类                               
2025-10-24 手机        7    0   0    nan%  0.00%
2025-10-25 手机        9    0   0    nan%  0.00%
2025-10-27 手机        9    1   0   0.00%  0.00%
2025-10-28 手机        3    0   0    nan%  0.00%
2025-10-29 手机       20    6   1  16.67%  5.00%
2025-10-30 手机       27    9   0   0.00%  0.00%
2025-10-31 手机       46    9   2  22.22%  4.35%
2025-11-01 手机       52   12   0   0.00%  0.00%
           电脑        1    1   0   0.00%  0.00%
2025-11-02 手机       54   14   1   7.14%  1.85%
           电脑        1    1   0   0.00%  0.00%
2025-11-03 手机       57   14   0   0.00%  0.00%
           电脑        1    0   0    nan%  0.00%
2025-11-04 手机       42   12   1   8.33%  2.38%
           电脑        2    1   0   0.00%  0.00%
2025-11-05 手机       47   18   0   0.00%  0.00%
2025-11-06 手机       50   12   0   0.00%  0.00%
2025-11-07 手机       46    8   0   0.00%  0.00%
2025-11-08 手机       67   16   0   0.00%  0.00%
2025-11-09 手机       59   11   0   0.00%  0.00%
2025-11-10 手机       65   12   1   8.33%  1.54%
2025-11-11 手机       41    6   1  16.67%  2.44%
2025-11-12 手机       20    2   0   0.00%  0.00%
2025-11-13 手机       19    2   0   0.00%  0.00%
2025-11-14 手机       12    4   0   0.00%  0.00%
2025-11-15 手机       12    4   0   0.00%  0.00%
2025-11-16 手机       11    3   0   0.00%  0.00%
2025-11-17 手机        9    2   0   0.00%  0.00%

In [94]:
# df[df.order_id==4190031][['order_id', '下单日期', '来源渠道','设备种类','是否进件','是否出库']]

# 转化数据

In [95]:
# df_gpc = df[df['来源渠道']=='付费灯火-高频词']
# df_gpc_group = group_data(df_gpc)
# df_gpc_group = df_gpc_group[['去重订单数', '前置拦截', '拦截率', '是否进件', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
#                 , '客户取消', '无法联系', '出库前风控强拒', '待审核', '是否出库', '进件出库率', '订单出库率']]
# df_gpc_group

In [96]:
# df_lhc = df[df['来源渠道']=='付费灯火-蓝海词']
# df_lhc_group = group_data(df_lhc)
# df_lhc_group = df_lhc_group[['去重订单数', '前置拦截', '拦截率', '是否进件', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
#                 , '客户取消', '无法联系', '出库前风控强拒', '待审核', '是否出库', '进件出库率', '订单出库率']]
# df_lhc_group

In [97]:
# df_apple = df[df['归属渠道']=='付费邦道-苹果旗舰']
# df_apple_group = group_data(df_apple)
# df_apple_group = df_apple_group[['去重订单数', '前置拦截', '拦截率', '是否进件', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
#                 , '客户取消', '无法联系', '出库前风控强拒', '待审核', '是否出库', '进件出库率', '订单出库率']]
# df_apple_group
# df_ss1.columns

In [98]:
# df_ss1 = df[df['来源渠道']=='搜索侠客行-首页']
# df_ss1_group = group_data(df_ss1)
# df_ss1_group = df_ss1_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
#                 , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
# df_ss1_group

In [99]:
# df_ss1['取消原因2'] = df_ss1['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_ss1_split = df_ss1[df_ss1['取消原因2'].notna() & (df_ss1['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
# # df_ss1_split.取消原因2.value_counts
# df_ss1_quyy = df_ss1[((df_ss1['机审强拒'] == 1)|(df_ss1['人审拒绝']==1)|(df_ss1['出库前风控强拒']==1))&(df_ss1['客户取消']==0)]
# df_ss1_split = df_ss1_quyy[df_ss1['取消原因2'].notna() & (df_ss1['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
# df_ss1_split

In [100]:
# df_ss1_split = df_ss1[df_ss1['拒绝理由'].notna() & (df_ss1['拒绝理由'].str.strip() != '')][['下单日期', 'order_number', '拒绝理由']]
# df_ss1_split

In [101]:
# df_ss2 = df[df['来源渠道']=='搜索侠客行-核心词']
# df_ss2_group = group_data(df_ss2)
# df_ss2_group = df_ss2_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
#                 , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
# df_ss2_group

In [102]:
# df_ss2_quyy = df_ss2[((df_ss2['机审强拒'] == 1)|(df_ss2['人审拒绝']==1)|(df_ss2['出库前风控强拒']==1))&(df_ss2['客户取消']==0)]
# df_ss2_split = df_ss2_quyy[df_ss2['取消原因2'].notna() & (df_ss2['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
# df_ss2_split

In [103]:
# df.columns
# # df_ss2_split = df_ss2[df_ss2['拒绝理由'].notna()][['下单日期', 'order_number', '拒绝理由']]
# # 过滤掉 NaN、空字符串和纯空白字符
# df_ss2_split = df_ss2[df_ss2['拒绝理由'].notna() & (df_ss2['拒绝理由'].str.strip() != '')][['下单日期', 'order_number', '拒绝理由']]
# df_ss2_split

In [104]:
# df[df['来源渠道']=='搜索邦道-小程序推广'].columns

In [105]:
# df_xsxs = df[df['来源渠道']=='搜索邦道-销售线索']
# df_xsxs_group = group_data(df_xsxs)
# df_xsxs_group = df_xsxs_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
#                 , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
# df_xsxs_group

In [106]:
# df_xsxs_quyy = df_xsxs[((df_xsxs['机审强拒'] == 1)|(df_xsxs['人审拒绝']==1)|(df_xsxs['出库前风控强拒']==1))&(df_xsxs['客户取消']==0)]
# df_xsxs_split = df_xsxs_quyy[df_xsxs['取消原因2'].notna() & (df_xsxs['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
# df_xsxs_split

In [107]:
# df_xsxs_split = df_xsxs[df_xsxs['拒绝理由'].notna() & (df_xsxs['拒绝理由'].str.strip() != '')][['下单日期', 'order_number', '拒绝理由']]
# df_xsxs_split

In [108]:
# df_xcx = df[df['来源渠道']=='搜索邦道-小程序推广']
# df_xcx_group = group_data(df_xcx)
# df_xcx_group = df_xcx_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
#                 , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
# df_xcx_group

In [109]:
# df_xcx_split = df_xcx[df_xcx['拒绝理由'].notna() & (df_xcx['拒绝理由'].str.strip() != '')][['下单日期', 'order_number', '拒绝理由']]
# df_xcx_split

In [110]:
# df_xcx_quyy = df_xcx[((df_xcx['机审强拒'] == 1)|(df_xcx['人审拒绝']==1)|(df_xcx['出库前风控强拒']==1))&(df_xcx['客户取消']==0)]
# df_xcx_split = df_xcx_quyy[df_xcx['取消原因2'].notna() & (df_xcx['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
# df_xcx_split

# 高频词、蓝海词

In [111]:
# #搜索灯火蓝海词进件明细
# df_ssdhj_lhc=df_j[df_j["来源渠道"]=="付费灯火-蓝海词"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道","订单状态","审核状态","拒绝理由","取消原因","电审拒绝原因"]]
# #搜索灯火蓝海词出库明细,阿钗需求
# df_ssdhck_lhc=df_cks[df_cks["来源渠道"]=="付费灯火-蓝海词"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# #搜索灯火高频词进件明细,阿钗需求
# df_ssdhj_gpc=df_j[df_j["来源渠道"]=="付费灯火-高频词"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道","订单状态","审核状态","拒绝理由","取消原因","电审拒绝原因"]]
# #搜索灯火高频词出库明细,阿钗需求
# df_ssdhck_gpc=df_cks[df_cks["来源渠道"]=="付费灯火-高频词"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# with pd.ExcelWriter(f'F:/灯火进件、出库明细/{Today1}_搜索灯火.xlsx', engine='openpyxl') as writer:
#     df_ssdhj_lhc.to_excel(writer, sheet_name='蓝海词进件明细', index=False)
# with pd.ExcelWriter(f'F:/灯火进件、出库明细/{Today1}_搜索灯火.xlsx', engine='openpyxl', mode='a') as writer:
#     df_ssdhck_lhc.to_excel(writer, sheet_name='蓝海词出库明细', index=False)
#     df_ssdhj_gpc.to_excel(writer, sheet_name='高频词进件明细', index=False)
#     df_ssdhck_gpc.to_excel(writer, sheet_name='高频词出库明细', index=False)

In [112]:
# #付费邦道-首页进件明细
# df_ffbdsy=df_j[df_j["来源渠道"]=="付费邦道-首页"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# #付费邦道-首页出库明细
# df_ffbdsyck=df_cks[df_cks["来源渠道"]=="付费邦道-首页"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# #邦道-自定义进件明细
# df_bdzdy=df_j[df_j["来源渠道"]=="邦道-自定义"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# #邦道-自定义出库明细
# df_bdzdyck=df_cks[df_cks["来源渠道"]=="邦道-自定义"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# #邦道-销售线索进件明细
# df_xsxs=df_j[df_j["来源渠道"]=="邦道-销售线索"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# #邦道-销售线索出库明细
# df_xsxsck=df_cks[df_cks["来源渠道"]=="邦道-销售线索"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# #付费侠客行-苹果旗舰进件明细
# df_xkx=df_j[df_j["来源渠道"]=="付费侠客行-苹果旗舰"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# #付费侠客行-苹果旗舰出库明细
# df_xkxck=df_cks[df_cks["来源渠道"]=="付费侠客行-苹果旗舰"][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# with pd.ExcelWriter(f'F:/灯火进件、出库明细/{Today1}_邦道.xlsx', engine='openpyxl') as writer:
#     df_ffbdsy.to_excel(writer, sheet_name='付费邦道-首页进件明细', index=False)
# with pd.ExcelWriter(f'F:/灯火进件、出库明细/{Today1}_邦道.xlsx', engine='openpyxl', mode='a') as writer:
#     df_ffbdsyck.to_excel(writer, sheet_name='付费邦道-首页出库明细', index=False)
#     df_bdzdy.to_excel(writer, sheet_name='邦道-自定义进件明细', index=False)
#     df_xkx.to_excel(writer, sheet_name='付费侠客行-苹果旗舰进件明细', index=False)
#     df_xkxck.to_excel(writer, sheet_name='付费侠客行-苹果旗舰出库明细', index=False)
#     df_bdzdyck.to_excel(writer, sheet_name='邦道-自定义出库明细', index=False)
#     df_xsxs.to_excel(writer, sheet_name='邦道-销售线索进件明细', index=False)
#     df_xsxsck.to_excel(writer, sheet_name='邦道-销售线索出库明细', index=False)

In [113]:
# qd_list = ["付费侠客行-苹果旗舰", "付费邦道-苹果旗舰", "付费邦道-华为专区", "付费邦道-降价了","付费邦道-智投","付费邦道-搜索","付费邦道-首页","付费灯火-蓝海词","付费灯火-高频词","付费流量-灯火内测"]
# df_ck_all = df_cks[df_cks["来源渠道"].isin(qd_list)][["下单日期","订单号","商品型号","总租金","买断金额","租赁方案","来源渠道"]]
# df_ck_all.to_excel('F:/需求/总需求/搜索灯火.xlsx',index=False)

# 搜索渠道

In [114]:
# # df_ss = df[df.归属渠道=='搜索渠道']
# # df_ss_group = df_ss.groupby('下单日期').agg({'是否进件': 'sum', '是否出库': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库':'出库'})
# # df_ss_group
# # with pd.ExcelWriter(f'F:/灯火进件、出库明细/{Today1}_搜索渠道.xlsx', engine='openpyxl') as writer:
# #     df_ss_group.to_excel(writer, sheet_name='搜索渠道')
# df_order = query(sql1)
# # 数据处理
# df = process_data(df_order)

# # 获取各个渠道的数据
# df[df['来源渠道']=='搜索侠客行-首页']


In [115]:
# df_ss1 = df[df['来源渠道']=='搜索侠客行-首页']
# df_ss1_group = group_data(df_ss1, '下单日期')
# df_ss1_group

# 定时任务

In [6]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler

hour = 10
minute = 50
# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job1():
    
    # print(f'执行定时任务：现在是每日的{H}点{int(M)+1}分')
    path = r'\\digua\迪瓜租机\20.搜索灯火/'
    # path = f'F:\需求\总需求/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    print(f'执行定时任务：现在是{Today1}的50分')
    # 查询数据
    df_order = query(sql1)
    # 数据处理
    df = process_data(df_order)
    
    # 获取各个渠道的数据
    # df_ss1 = df[df['来源渠道']=='搜索侠客行-首页']
    df_ss2 = df[df['来源渠道']=='搜索侠客行-核心词']
    # df_xsxs = df[df['来源渠道']=='搜索邦道-销售线索']
    df_xcx = df[df['来源渠道']=='搜索邦道-小程序推广']
    df_ss_iphone17 = df[df['来源渠道']=='搜索侠客行-苹果17']
    # df_ss_qs = df[df['来源渠道']=='广州麒升-苹果17']
    df_ss_pgxc = df[df['来源渠道']=='搜索侠客行-苹果新词']
    
    xkx_list = ['搜索侠客行-首页', '搜索侠客行-核心词', '搜索侠客行-苹果17','搜索侠客行-苹果新词']
    bd_list = ['搜索邦道-销售线索', '搜索邦道-小程序推广']
    df_xkx = df[df['来源渠道'].isin(xkx_list)]
    df_bd = df[df['来源渠道'].isin(bd_list)]
    
    # 处理分组数据
    # df_ss1_group = group_data(df_ss1, '下单日期')
    # df_ss1_group = df_ss1_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
    #             , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
    df_ss2_group = group_data(df_ss2, '下单日期')
    df_ss2_group = df_ss2_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
                , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
    # df_xsxs_group = group_data(df_xsxs, '下单日期')
    # df_xsxs_group = df_xsxs_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
    #                 , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
    df_xcx_group = group_data(df_xcx, '下单日期')
    df_xcx_group = df_xcx_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
                    , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
    df_ss_iphone17_group = group_data(df_ss_iphone17, '下单日期')
    df_ss_iphone17_group = df_ss_iphone17_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
                    , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
    
    df_ss_pgxc_group = group_data(df_ss_pgxc, '下单日期')
    df_ss_pgxc_group = df_ss_pgxc_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
                    , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
    
    # df_ss_qs_group = group_data(df_ss_qs, '下单日期')
    # df_ss_qs_group = df_ss_qs_group[['去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率'
    #                 , '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '订单出库率']]
    
    df_xkx_group = group_data(df_xkx, '下单日期')
    df_xkx_group = df_xkx_group[['去重订单数', '进件数', '风控通过件', '出库', '风控通过率', '进件出库率', '订单出库率']]
    df_bd_group = group_data(df_bd, '下单日期')
    df_bd_group = df_bd_group[['去重订单数', '进件数', '风控通过件', '出库', '风控通过率', '进件出库率', '订单出库率']]
    # df_qs_group = group_data(df_ss_qs, '下单日期')
    # df_qs_group = df_qs_group[['去重订单数', '进件数', '风控通过件', '出库', '风控通过率', '进件出库率', '订单出库率']]
    
    
    
    
    

    # 处理拒绝原因数据
    # 处理拒绝原因数据
    # df_ss1_quyy = df_ss1[((df_ss1['机审强拒'] == 1)|(df_ss1['人审拒绝']==1)|(df_ss1['出库前风控强拒']==1))&(df_ss1['客户取消']==0)]
    # df_ss1_split = df_ss1_quyy[df_ss1['取消原因2'].notna() & (df_ss1['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
    df_ss2_quyy = df_ss2[((df_ss2['机审强拒'] == 1)|(df_ss2['人审拒绝']==1)|(df_ss2['出库前风控强拒']==1))&(df_ss2['客户取消']==0)]
    df_ss2_split = df_ss2_quyy[df_ss2['取消原因2'].notna() & (df_ss2['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
    # df_xsxs_quyy = df_xsxs[((df_xsxs['机审强拒'] == 1)|(df_xsxs['人审拒绝']==1)|(df_xsxs['出库前风控强拒']==1))&(df_xsxs['客户取消']==0)]
    # df_xsxs_split = df_xsxs_quyy[df_xsxs['取消原因2'].notna() & (df_xsxs['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
    df_xcx_quyy = df_xcx[((df_xcx['机审强拒'] == 1)|(df_xcx['人审拒绝']==1)|(df_xcx['出库前风控强拒']==1))&(df_xcx['客户取消']==0)]
    df_xcx_split = df_xcx_quyy[df_xcx['取消原因2'].notna() & (df_xcx['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
    df_ss_iphone17_quyy = df_ss_iphone17[((df_ss_iphone17['机审强拒'] == 1)|(df_ss_iphone17['人审拒绝']==1)|(df_ss_iphone17['出库前风控强拒']==1))&(df_ss_iphone17['客户取消']==0)]
    df_ss_iphone17_split = df_ss_iphone17_quyy[df_ss_iphone17['取消原因2'].notna() & (df_ss_iphone17['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
    
    # df_ss_qs_quyy = df_ss_qs[((df_ss_qs['机审强拒'] == 1)|(df_ss_qs['人审拒绝']==1)|(df_ss_qs['出库前风控强拒']==1))&(df_ss_qs['客户取消']==0)]
    # df_ss_qs_split = df_ss_qs_quyy[df_ss_qs['取消原因2'].notna() & (df_ss_qs['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]
    df_ss_pgxc_quyy = df_ss_pgxc[((df_ss_pgxc['机审强拒'] == 1)|(df_ss_pgxc['人审拒绝']==1)|(df_ss_pgxc['出库前风控强拒']==1))&(df_ss_pgxc['客户取消']==0)]
    df_ss_pgxc_split = df_ss_pgxc_quyy[df_ss_pgxc['取消原因2'].notna() & (df_ss_pgxc['取消原因2'].str.strip() != '')][['下单日期', 'order_number', '取消原因2']]

    # 搜索侠客行-首页、搜索侠客行-核心词、搜索侠客行-苹果17 
    # 机型内存的进件数、出库、进件占比、出库的占比
    # df_iphone_all.loc[:, '出库占比'] = df_iphone_all['出库'] / df_iphone_all['出库'].sum()
    # df_iphone_all.loc[:, '出库占比'] = df_iphone_all['出库占比'].apply(lambda x:format(x,".2%"))
    # df_iphone_all_new = df_iphone_all.reset_index()
    # df_iphone_all_new.loc['合计'] = df_iphone_all_new.select_dtypes(include=[np.number]).sum()
    
    # df_ss1_product = df_ss1.groupby('机型内存').agg({'是否进件': 'sum', '是否出库': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库':'出库'})
    # df_ss1_product.loc[:,'进件占比'] = (df_ss1_product['进件数']/df_ss1_product['进件数'].sum()).apply(lambda x:format(x,".2%"))
    # df_ss1_product.loc[:,'出库占比'] = (df_ss1_product['出库']/df_ss1_product['出库'].sum()).apply(lambda x:format(x,".2%"))
    # # df_ss1_product_new = df_ss1_product.reset_index()
    # df_ss1_product.loc['合计'] = df_ss1_product.select_dtypes(include=[np.number]).sum()
    
    df_ss2_product = df_ss2.groupby('机型内存').agg({'是否进件': 'sum', '是否出库': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库':'出库'})
    df_ss2_product.loc[:,'进件占比'] = (df_ss2_product['进件数']/df_ss2_product['进件数'].sum()).apply(lambda x:format(x,".2%"))
    df_ss2_product.loc[:,'出库占比'] = (df_ss2_product['出库']/df_ss2_product['出库'].sum()).apply(lambda x:format(x,".2%"))
    # df_ss2_product_new = df_ss2_product.reset_index()
    df_ss2_product.loc['合计'] = df_ss2_product.select_dtypes(include=[np.number]).sum()
    
    df_ss_iphone17_product = df_ss_iphone17.groupby('机型内存').agg({'是否进件': 'sum', '是否出库': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库':'出库'})
    df_ss_iphone17_product.loc[:,'进件占比'] = (df_ss_iphone17_product['进件数']/df_ss_iphone17_product['进件数'].sum()).apply(lambda x:format(x,".2%"))
    df_ss_iphone17_product.loc[:,'出库占比'] = (df_ss_iphone17_product['出库']/df_ss_iphone17_product['出库'].sum()).apply(lambda x:format(x,".2%"))
    # df_ss_iphone17_product_new = df_ss_iphone17_product.reset_index()
    df_ss_iphone17_product.loc['合计'] = df_ss_iphone17_product.select_dtypes(include=[np.number]).sum()
    

    with pd.ExcelWriter(path + f'搜索侠客行{Today1}.xlsx', engine='openpyxl') as writer:
        # df_ss1_group.to_excel(writer, sheet_name='搜索侠客行-首页')
        df_ss2_group.to_excel(writer, sheet_name='搜索侠客行-核心词')
        df_ss_pgxc_group.to_excel(writer, sheet_name='搜索侠客行-苹果新词')
        # df_xsxs_group.to_excel(writer, sheet_name='搜索邦道-销售线索')
        df_xcx_group.to_excel(writer, sheet_name='搜索邦道-小程序推广')
        df_ss_iphone17_group.to_excel(writer, sheet_name='搜索侠客行-苹果17')
        # df_ss_qs_group.to_excel(writer, sheet_name='广州麒升-苹果17')
        
        
        df_xkx_group.to_excel(writer, sheet_name='搜索侠客行-汇总')
        df_bd_group.to_excel(writer, sheet_name='搜索邦道-汇总')
        # df_qs_group.to_excel(writer, sheet_name='广州麒升-苹果17-汇总')
        
        # df_ss1_split.to_excel(writer, sheet_name='搜索侠客行-首页_拒绝原因', index=False)
        df_ss2_split.to_excel(writer, sheet_name='搜索侠客行-核心词_拒绝原因', index=False)
        df_ss_pgxc_split.to_excel(writer, sheet_name='搜索侠客行-苹果新词_拒绝原因', index=False)
        
        # df_xsxs_split.to_excel(writer, sheet_name='搜索邦道-销售线索_拒绝原因', index=False)
        df_xcx_split.to_excel(writer, sheet_name='搜索邦道-小程序推广_拒绝原因', index=False)
        df_ss_iphone17_split.to_excel(writer, sheet_name='搜索侠客行-苹果17_拒绝原因', index=False)
        # df_ss_qs_split.to_excel(writer, sheet_name='广州麒升-苹果17_拒绝原因', index=False)
        
        # df_ss1_product.to_excel(writer, sheet_name='搜索侠客行-首页_机型内存', index=True)
        df_ss2_product.to_excel(writer, sheet_name='搜索侠客行-核心词_机型内存', index=True)
        df_ss_iphone17_product.to_excel(writer, sheet_name='搜索侠客行-苹果17_机型内存', index=True)
    del df_ss2_group,df_ss2_split,df_xcx_group,df_xcx_split,df_ss_iphone17_group,df_ss_iphone17_split,df_xkx_group,df_bd_group, df_ss2_product, df_ss_iphone17_product
    gc.collect()# df_ss1_group,,df_ss1_split,df_xsxs_group,df_xsxs_split, df_ss1_product
    print("回收内存执行完毕！\n")
scheduler1 = BackgroundScheduler()
# 添加一个cron任务，每天的10:50执行

# my_job1()
# try:
#     scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute, args=[df_ss1_group, df_ss2_group, df_ss1_split, df_ss2_split])
# except:
#     time.sleep(60)
#     scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute+1, args=[df_ss1_group, df_ss2_group, df_ss1_split, df_ss2_split])
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)
scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# 启动scheduler1
scheduler1.start()


# 索引